## Importing the needed libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

## Loading the training CV data

In [ ]:
# Loading the processed IMU data 
IMU_data = pd.read_csv('data/training.csv')

# Previewing the IMU data
IMU_data.head()

In [ ]:
feature_cols = 'axdt_var,aydt_var,gzdt_var'.split(',')

In [ ]:
# Separating the features (IMU data)
X = IMU_data[[col for col in IMU_data.columns if col !='motion_state']].copy()
X = IMU_data[feature_cols]
X.head()

In [ ]:
y = IMU_data['motion_state'].copy()
y.head()

In [ ]:
# Splitting data into testing and training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

## Setting up the Pipeline and Parameter Grid Search

In [ ]:
# Setting up pipeline to test different scaling techniques with SVM
pipe = Pipeline([('preprocessor', StandardScaler()),
                 ('classifier', [SVC()])])

In [ ]:
# Setting up the parameter grid to tune the hyperparameters and get the best results
param_grid = [{'preprocessor': [StandardScaler(), RobustScaler()],
               'classifier': [SVC()],
               'classifier__C': [0.001, 0.01, 0.1, 1 ,10],
               'classifier__gamma': [0.001, 0.01, 0.1,1],
               'classifier__kernel': ['rbf','poly']
              }]

In [ ]:
# Performing a grid search across all the set hyperparameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

## Grid Search Best Results

In [ ]:
# Best parameters
print("Best Parameters:")
grid.best_params_

In [ ]:
# Best cross-validation train score
print("Best CV Train Score:")
grid.cv_results_['mean_train_score'][grid.best_index_]

## Performance on Testing Data

In [ ]:
# Test set accuracy
best_test_model = grid.best_estimator_
print("Test-set score:")
best_test_model.score(X_test, y_test)

In [ ]:
# Testing on completely new dataset
IMU_data2 = pd.read_csv('data/testing.csv')
X2 = IMU_data2[[col for col in IMU_data.columns if col !='motion_state']].copy()
X2 = IMU_data2[feature_cols]
y2 = IMU_data2['motion_state'].copy()
print("Testing 2 set score:")
best_test_model.score(X2, y2)

In [ ]:
# Testing on completely new dataset
IMU_data3 = pd.read_csv('data/testing2.csv')
X3 = IMU_data3[[col for col in IMU_data.columns if col !='motion_state']].copy()
X3 = IMU_data3[feature_cols]
y3 = IMU_data3['motion_state'].copy()
print("Testing 3 set score:")
best_test_model.score(X3, y3)

## Visualizing Accuracy

In [ ]:
# Function for outputting the confusion matrix
def evaluate_classification(true_values, predicted_values, ticklabels, model):
    # Calculate confusion matrix
    confusion_mat = confusion_matrix(true_values, predicted_values)
    
    # Plot heatmap
    sns.heatmap(confusion_mat, xticklabels=ticklabels, yticklabels=ticklabels, annot=True, fmt='d', cbar=False, cmap='viridis')
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Confusion Matrix - ' + model)
    plt.show()

    # Calculate scores
    accuracy = accuracy_score(true_values, predicted_values)
    precision = precision_score(true_values, predicted_values, average='macro')
    recall = recall_score(true_values, predicted_values, average='macro')
    f1 = f1_score(true_values, predicted_values, average='macro')

    '''# Calculate additional metrics
    tn, fp, fn, tp = confusion_mat.ravel()

    # Print additional metrics
    print("Metrics for", model)
    print("True Negatives:", tn)
    print("False Positives:", fp)
    print("False Negatives:", fn)
    print("True Positives:", tp)'''

    # Print classification report
    print("\nClassification Report for", model + ":")
    print(classification_report(true_values, predicted_values))

    # Print scores
    print("\nAccuracy Score:", accuracy)
    print("Precision Score:", precision)
    print("Recall Score:", recall)
    print("F1 Score:", f1)
    
pred_val = grid.predict(X_test)
evaluate_classification(y_test, pred_val, ['rest', 'straight', 'turn'], 'Support Vector Classifier')

In [ ]:
pred_val = grid.predict(X2)
evaluate_classification(y2, pred_val, ['rest', 'straight', 'turn'], 'Support Vector Classifier')

In [ ]:
pred_val = grid.predict(X3)
evaluate_classification(y3, pred_val, ['rest', 'straight', 'turn'], 'Support Vector Classifier')